<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
🗺️ תהליך הסבת שכבת תיקוני 97 ב' לבנק"ל מודרני 🗺️ 

In [ ]:
print('7. Amendment 97B borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from configs import CNFG
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  7.a Reading inputs')

<h1 style="text-align:right;font-size:100%">
טבלת ניהול תהליכים של הסדר קרקעות

In [ ]:
hek_df_cols = ['מספר פניה','תאריך בקשה','גוש','תת גוש','חלקה','תאריך שליחה לפקיד הסדר','תאריך אישור הממונה על המרשם','תאריך אישור הרשם',' שלב עבודה נוכחי','תאריך התחלה','תאריך סיום בפועל']
date_cols = ['תאריך בקשה', 'תאריך שליחה לפקיד הסדר', 'תאריך אישור הממונה על המרשם', 'תאריך אישור הרשם']

hek_df = pd.read_csv(CNFG.Local + '97b_nuhil_tahalichim.csv',
                     usecols = hek_df_cols,
                     encoding = 'cp1255',
                     parse_dates = date_cols)

hek_df.sort_values('מספר פניה', inplace=True)


del hek_df_cols
print('    input 1/5 Read')

<h1 style="text-align:right;font-size:100%">
'שכבת תיקוני 97ב

In [ ]:
amendment97B_cols = ['MISPAR_PNIA', 'GUSH_NUM', 'GUSH_SUFFIX', 'PARCELS', 'geometry']

amendment97B_df = gpd.read_file(CNFG.CADSDE + 'Tikun_Hesder_97B.gdb',
                                layer = 'Tikun_Hesder_97B',
                                include_fields = amendment97B_cols)


del amendment97B_cols
print('    input 2/5 Read')

<h1 style="text-align:right;font-size:100%">
שכבת חלקות קיימות

In [ ]:
parcels_cols = ['PARCEL', 'GUSH_NUM', 'GUSH_SUFFIX', 'geometry']

parcels_df = gpd.read_file(CNFG.Arcstorm + 'PARCEL_ALL.gdb', layer = 'PARCEL_ALL', include_fields = parcels_cols)

parcels_df = parcels_df.to_crs(2039)

del parcels_cols
print('    input 3/5 Read')

<h1 style="text-align:right;font-size:100%">
שכבת חלקות היסטוריות

In [ ]:
parcels_history_cols = ['PARCEL', 'GUSH_NUM', 'GUSH_SUFFIX', 'geometry']

parcels_history_df = gpd.read_file(CNFG.CADSDE + 'PARCEL_HISTORY_HASAVA.gdb',
                                   layer = 'PARCEL_HISTORY_HASAVA',
                                   include_fields = parcels_history_cols)

del parcels_history_cols
print('    input 4/5 Read')

<h1 style="text-align:right;font-size:100%">
טבלת מספרי זיהוי של גושים

In [ ]:
Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['Name', 'BlockUniqueID'])
print('    input 5/5 Read')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  7.b Cleaning data')

<h1 style="text-align:right;font-size:125%">
:יצירת שכבת חלקות כללית
<h1 style="text-align:right;font-size:100%">
יצירת שדה חדש לזיהוי חלקה לפי מספרה, מספר הגוש, ומסתר התת גוש כשדה טקסט
<h1 style="text-align:right;font-size:100%">
יצירת שכבה חדשה
<h1 style="text-align:right;font-size:100%">
הסרת כפילויות

In [ ]:
parcels_df['parcel_name'] = parcels_df['GUSH_NUM'].astype(str) + '-' + parcels_df['GUSH_SUFFIX'].astype(str) + '-' + parcels_df['PARCEL'].astype(str)
parcels_history_df['parcel_name'] = parcels_history_df['GUSH_NUM'].astype(str) + '-' + parcels_history_df['GUSH_SUFFIX'].astype(str) + '-' + parcels_history_df['PARCEL'].astype(str)

parcels_deep = pd.concat([parcels_df, parcels_history_df], ignore_index = False)
parcels_deep = parcels_deep.sort_values(['GUSH_NUM', 'GUSH_SUFFIX', 'PARCEL'], ignore_index = True)
parcels_deep.drop_duplicates(keep = 'first', inplace = True)

del [parcels_df, parcels_history_df]

<h1 style="text-align:right;font-size:125%">
:עיבוד טבלת ניהול תהליכים של הסדר קרקעות
<h1 style="text-align:right;font-size:100%">
הסרת רשומות עם ערכי חלקה לא חוקיים (אלו מקרים של בדיקה טבנית במערכת ניהול תהליכים)
<h1 style="text-align:right;font-size:100%">
יצירת שדה לסטטוס נוכחי ולתאריך שלו
<h1 style="text-align:right;font-size:100%">
יצירת שדה של כלל חלקות לפי מספר פניה
<h1 style="text-align:right;font-size:100%">
השלמת ערכים חסרים לשדה תת-גוש
<h1 style="text-align:right;font-size:100%">
יצירת שדה חדש לזיהוי חלקה לפי מספרה, מספר הגוש, ומסתר התת גוש כשדה טקסט

In [ ]:
print('  7.c Data conversion')

In [ ]:
hek_df = hek_df[~hek_df['חלקה'].isin([9999.0, 999.0])]

hek_df['תאריך בקשה'] = pd.to_datetime(hek_df['תאריך בקשה'], format='%d/%m/%Y')
hek_df['recent_date'] = hek_df[date_cols].max(axis = 1)
s4 = hek_df['recent_date'].eq(hek_df['תאריך אישור הרשם'])
s3 = hek_df['recent_date'].eq(hek_df['תאריך אישור הממונה על המרשם'])
s2 = hek_df['recent_date'].eq(hek_df['תאריך שליחה לפקיד הסדר'])
s1 = hek_df['recent_date'].eq(hek_df['תאריך בקשה'])

hek_df['status_name'] = np.select(condlist = (s4, s3, s2, s1),
                                  choicelist = ('אישור הרשם', 'אישור הממונה על הרשם', 'שליחה לפקיד הסדר', 'בקשה'),
                                  default = 'בקשה')


hek_status_clf = {'בקשה':14,
                  'שליחה לפקיד הסדר':13,
                  'אישור הממונה על הרשם':13,
                  'אישור הרשם':13}

hek_df['status'] = hek_df['status_name'].map(hek_status_clf)
hek_df['status'] = hek_df['status'].astype('Int32')



hek_df['חלקות'] = hek_df.astype(str).groupby(['מספר פניה'])['חלקה'].transform(','.join)


hek_df['תת גוש'] = hek_df['תת גוש'].fillna(0).astype(int)


hek_df['parcel_name'] = hek_df['גוש'].astype(str) + '-' + hek_df['תת גוש'].astype(str) + '-' + hek_df['חלקה'].astype(str)

<h1 style="text-align:right;font-size:125%">
:יצירת שכבה של טבלת ניהול תהליכים
<h1 style="text-align:right;font-size:100%">
מיזוג הטבלה עם שכבת החלקות הכוללת שנוצרה בסעיף קודם
<h1 style="text-align:right;font-size:100%">
קיבוץ מרחבי לפי מספר פניה

In [ ]:
hek_df = hek_df.merge(parcels_deep[['geometry', 'parcel_name']], on = 'parcel_name', how = 'left')

hek_gdf = gpd.GeoDataFrame(hek_df, geometry = 'geometry', crs = 2039)

hek_gdf_dissolve = hek_gdf.dissolve(by='מספר פניה', aggfunc='max', as_index=False, dropna=False)

del [hek_df, hek_gdf, parcels_deep]

<h1 style="text-align:right;font-size:125%">
:איחוד עם שכבת תיקוני 97ב
<h1 style="text-align:right;font-size:100%">
הוספת רשומות שכבת 97ב אל שכבת ניהול תהליכים שנוצרה בסעיף קודם
<h1 style="text-align:right;font-size:100%">
סידור שמות של שדות
<h1 style="text-align:right;font-size:100%">
מילוי ערכים חסרים והגדרת סוג השדות (במידת הצורך)    
<h1 style="text-align:right;font-size:100%">
מחיקת שדות שאינם רלוונטיים יותר       
<h1 style="text-align:right;font-size:100%">
מיון לפי מספר פניה ותאריך בקשה            
<h1 style="text-align:right;font-size:100%">
הסרת כפילויות                

In [ ]:
appended = pd.concat([hek_gdf_dissolve, amendment97B_df], ignore_index=True)


appended.rename(columns={'תאריך בקשה'                  : 'requestDate',
                         'תאריך אישור הממונה על המרשם' : 'appovalDate',
                         'תאריך אישור הרשם'            : 'registrationDate'},
                inplace=True)


appended['MISPAR_PNIA'].fillna(hek_gdf_dissolve['מספר פניה'], inplace=True)
appended['MISPAR_PNIA'] = appended['MISPAR_PNIA'].astype(int)


appended['PARCELS'].fillna(hek_gdf_dissolve['חלקות'], inplace=True)


appended['GUSH_NUM'].fillna(hek_gdf_dissolve['גוש'], inplace=True)
appended['GUSH_NUM'] = appended['GUSH_NUM'].astype(int)


appended['GUSH_SUFFIX'].fillna(hek_gdf_dissolve['תת גוש'], inplace=True)
appended['GUSH_SUFFIX'].fillna(0, inplace = True)
appended['GUSH_SUFFIX'] = appended['GUSH_SUFFIX'].astype(int)


appended['BlockName'] = appended['GUSH_NUM'].astype(str) + '/' + appended['GUSH_SUFFIX'].astype(str)


appended.drop(columns=['מספר פניה','חלקות', 'חלקה', 'גוש', 'תת גוש'], inplace=True)
appended.sort_values(['MISPAR_PNIA', 'requestDate'], inplace=True)
appended.drop_duplicates(subset='MISPAR_PNIA', keep='first', inplace=True)


del hek_gdf_dissolve

<h1 style="text-align:right;font-size:125%">
איחוד עם טבלת זיהןי גושים לקבלת מזהה גוש

In [ ]:
appended = appended.merge(Blocks, left_on = 'BlockName', right_on = 'Name', how = 'left')

del Blocks

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  7.d Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("CPBUniqueID", int),
                        ("BlockUniqueID", int),
                        ("ProcessType", int),
                        ("MapYear", int),
                        ("ProcessName", str),
                        ("GeodeticNetwork", int),    #לא קיים שדה לייבוא
                        ("Status", int),
                        #("PlanName", str),          #לא קיים שדה לייבוא
                        #("SurveyorLicenseID", int), #לא קיים שדה לייבוא
                        ("DataSource", int),
                        ("geometry", gpd.array.GeometryDtype)
                        ])


b97_new = gpd.GeoDataFrame(data = np.empty(0, dtype = new_columns),
                           geometry = 'geometry',
                           crs = 2039)

del new_columns

<h1 style="text-align:right;font-size:100%">
איכלוס שדות

In [ ]:
b97_new['geometry'] = appended['geometry']


b97_new['CPBUniqueID'] = appended['MISPAR_PNIA'] + 6000000


b97_new['ProcessName'] = appended['MISPAR_PNIA'].astype('str') + '/' + appended['BlockName'].astype('str')


b97_new['BlockUniqueID'] = appended['BlockUniqueID'].fillna(0)
b97_new['BlockUniqueID'] = b97_new['BlockUniqueID'].astype('Int64')


b97_new['ProcessType'] = 6


b97_new['MapYear'] = appended['appovalDate'].astype('datetime64[ns]').dt.year.astype('Int64')


b97_new['GeodeticNetwork'] = 0


b97_new['Status'] = appended['status'].fillna(14)
b97_new['Status'] = b97_new['Status'].astype('Int64')


#b97_new['PlanName'] = None


#b97_new['SurveyorLicenseID'] = None


b97_new['DataSource'] = np.where(~b97_new['geometry'].isna(), 5, 3)  #תהליך עם גיאומטריה מגיע מקובץ שרטוט, אחרת מגיע מדיגיטציה
b97_new['DataSource'] = b97_new['DataSource'].fillna(3)
b97_new['DataSource'] = b97_new['DataSource'].astype(int)

In [ ]:
b97_new.sort_values('CPBUniqueID', inplace=True)

b97_new = b97_new.to_crs(2039)

del appended

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  7.e Exporting results to product folder')

In [ ]:
b97_new_geom = b97_new[~b97_new['geometry'].isna()]
b97_new_no_geom = b97_new[b97_new['geometry'].isna()]

In [ ]:
no_geom_csv_dir = CNFG.CSVPath + 'Amendment97B_no_geom.csv'
json_dir        = CNFG.JsonPath + 'Amendment97B_geom.geojson'


b97_new_no_geom.drop(columns=['geometry']).to_csv(no_geom_csv_dir ,encoding = 'cp1255', index = False)


b97_new_geom.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([CNFG.BatchPath + 'Amendments97B-Save as Feature Class.bat']);